In [1]:
from IPython.core.display import display, HTML, Image
from time import sleep
from amazon.models import Review
from core.spiders import SplashBaseSpider
import pandas as pd
import yarl
import requests
from sklearn import decomposition, ensemble
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

ModuleNotFoundError: No module named 'pandas'

In [48]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    print(predictions)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [36]:
reviews = Review.objects.all()
train_df = pd.DataFrame()
labels = []
texts = []
gifts = []
notgifts = []

In [37]:
# Create a pandas dataframe from the results in order to analyse
#df = pd.DataFrame(list(Review.objects.all().values()))

In [38]:
#for review in reviews:
#    text = '{}.{}'.format(review.title, review.body)
#    texts.append(text)

In [39]:
#import csv
#with open('classifiers/training/gifts.csv', 'a') as f:
#    writer = csv.writer(f)
#    for text in notgifts:
#        writer.writerow([text, 'notgift'])
    

In [40]:
# Load the data
df = pd.read_csv('classifiers/training/gifts.csv', names=['text', 'label'])

In [41]:
df.head()

text label
0  It is a lovely colour and looks very sturdy, t...  gift
1  it is cute and fun, very pleased with it.bough...  gift
2  Happy with product..Bought as a gift and I hav...  gift
3  Baby Nessie.Lovely, cute gift, arrived before ...  gift
4  Pleasantly surprised!.Bought this as a gift. I...  gift

In [52]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['text'], df['label'])

In [43]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [44]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [45]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

## Naive Bayes

In [49]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: {}".format(accuracy))

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: {}".format(accuracy))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: {}".format(accuracy))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: {}".format(accuracy))

[0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0]
NB, Count Vectors: 0.9230769230769231
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0]
NB, WordLevel TF-IDF: 0.8846153846153846
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
NB, N-Gram Vectors: 0.7307692307692307
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0]
NB, CharLevel Vectors: 0.8076923076923077


# Linear Classifier

In [50]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: {}".format(accuracy))

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: {}".format(accuracy))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: {}".format(accuracy))

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: {}".format(accuracy))

[0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 1 0]
LR, Count Vectors: 0.8461538461538461
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0]
LR, WordLevel TF-IDF: 0.8461538461538461
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
LR, N-Gram Vectors: 0.7307692307692307
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0]
LR, CharLevel Vectors: 0.8076923076923077


## Random Forrest

In [15]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: {}".format(accuracy))

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: {}".format(accuracy))

RF, Count Vectors: 0.8846153846153846
RF, WordLevel TF-IDF: 0.7692307692307693


In [16]:
import xgboost
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: {}".format(accuracy))

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: {}".format(accuracy))

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: {}".format(accuracy))

Xgb, Count Vectors: 0.8461538461538461
Xgb, WordLevel TF-IDF: 0.8461538461538461
Xgb, CharLevel Vectors: 0.7692307692307693


## Use Linear Classifier

In [29]:
xtrain_count =  count_vect.transform(train_x)
classifier = linear_model.LogisticRegression()
classifier.fit(xtrain_count, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
texts = [review.body for review in reviews]
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(texts)
texts_count =  count_vect.transform(texts)

In [28]:
predictions = classifier.predict(texts_count)

ValueError: X has 1258 features per sample; expecting 728

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [57]:
text_clf = text_clf.fit(train_x, train_y)

In [67]:
predicted = text_clf.predict(texts)

In [70]:
for i,label in enumerate(predicted):
    print('{}: {}'.format(label, texts[i]))

1: I bought Nessie as a treat to myself, so that I could use some leaf teas that I had been given. I love the idea and it is well built, but I found filling and closing to be a bit of a chore given the relatively narrow/shallow nature of the holding chamber. I also found that it was hard to get water to circulate through the holes as air got trapped and blocked them. But, it did still make my tea so for that I give it a thumbs up!
1: i saw this and i just fell in love with it. im a big tea fan and recently got into the loose tea leaves. this is easy to clean and i love the design of it too. it is a bit on the pricey side but if you love your tea and dont want to have to loose leaves flowing into your mouth, i would highly recommend this product. ive seen other ones where they can hook into the side of the cup or mugs. i use a flask mug so i cant hook those designs onto my mug. because of nessies long neck, i have no problem of lifting it out when im done.
1: bought specifically since i

In [62]:
print(valid_x)

64     Worth a fiver.I bought this on the £5 special ...
94                                  Four Stars.Very good
45     ... - she hid it away from the children to enj...
55     looks interesting.looks great in presentation ...
61     Sweet box.This was  a gift.  the recipient was...
97                               Four Stars.Good variety
65     cute and works well for loose tea leaf drinker...
19     Great Value & Great Gift.Ive had about 7 of th...
40     I bought this as a present and it was sent ......
6      This product is of a poor quality and I would ...
37      Bought 2 for gifts, great product..Great product
79     Happy with product..Bought as a gift and I hav...
3      Baby Nessie.Lovely, cute gift, arrived before ...
12     Excellent selection of sweets and memories - g...
53     Good.Not as big as I was expecting. Our friend...
91     Fantastic!.A fab selection of retro sweets and...
31          Sweets Galore.Great selection, perfect gift!
1      it is cute and fun, very

In [66]:
print(p_df)

                                                  text
0    I bought Nessie as a treat to myself, so that ...
1    i saw this and i just fell in love with it. im...
2    bought specifically since it super easy to cle...
3    Absolutely adorable! Maybe not the best infuse...
4    As a tea strainer it is not that great, you ar...
5    Everything about this is amazing, I want every...
6    I love dino!  The holes are abit big so expect...
7    Was so excited as looks amazing but very disap...
8    very small but quite cute. It is a lovely colo...
9    This tea infuser is easy to use and to clean. ...
10                                              Lovely
11                                   works really well
12                 Very cute and makes a great present
13                                        Love it!!!!!
14              Really cute and easy to use and clean!
15   Its kind of cute I suppose, and works fairly w...
16   Cracking novelty item but a bit naff as a tea ...
17        